## Data frame preparation and cleaning for dashboard

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#1 - dataframe output from google vision
posts_df = pd.read_excel('https://raw.githubusercontent.com/scmartel/insy670/main/Google_Vision_DF.xlsx')

#2 - dataframe output from user posts with cleaning and mapping
map_df = pd.read_csv('https://raw.githubusercontent.com/scmartel/insy670/main/map_df.csv')

#3 - dataframe output from posts scrapped using comments
hashtag_df = pd.read_csv('https://raw.githubusercontent.com/scmartel/insy670/main/geocodes_comments2.csv')

In [ ]:
posts_df.head()

,Unnamed: 0,Caption,Comments,Likes,URL,Object_Num,Object_0_score,Object_1_score,Object_2_score,Object_3_score,Object_4_score,Object_5_score,Object_6_score,Object_7_score,Object_8_score,Object_9_score,Object_0,Object_1,Object_2,Object_3,Object_4,Object_5,Object_6,Object_7,Object_8,Object_9,Total_Objects_Names,Total_Objects_Scores,Label_Num,Label_0_score,Label_1_score,Label_2_score,Label_3_score,Label_4_score,Label_5_score,Label_6_score,Label_7_score,Label_8_score,Label_9_score,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5,Label_6,Label_7,Label_8,Label_9,Labels_Objects_Names,Labels_Objects_Scores,Engagement
0,0,“Always Stay Hungry” - work by @sasha.korban f...,3,175,https://scontent-atl3-1.cdninstagram.com/v/t51...,2,0.781665,0.530012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Person,Building,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Person', 'Building']","['0.7816652655601501', '0.5300119519233704']",10,0.964321,0.933414,0.921864,0.911200,0.893350,0.855671,0.851890,0.824412,0.819045,0.812318,Building,Car,White,Window,Infrastructure,Street fashion,Automotive lighting,Eyewear,Art,Neighbourhood,"['Building', 'Car', 'White', 'Window', 'Infras...","['0.964320719242096', '0.933414101600647', '0....",525
1,1,Work by @ronenglish on the famed Houston Bower...,2,115,https://scontent-atl3-1.cdninstagram.com/v/t51...,1,0.710405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Person'],['0.7104054093360901'],10,0.822278,0.821241,0.815670,0.809046,0.808544,0.774607,0.742931,0.730672,0.712364,0.680442,Painting,Art,Graffiti,Brick,Building,Facade,Mural,Rectangle,City,Illustration,"['Painting', 'Art', 'Graffiti', 'Brick', 'Buil...","['0.8222779035568237', '0.8212408423423767', '...",230
2,2,New work by @invaderwashere in Slovenia. 👾,2,111,https://scontent-atl3-1.cdninstagram.com/v/t51...,1,0.929607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2D barcode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2D barcode'],['0.9296074509620667'],10,0.867903,0.847361,0.824317,0.819373,0.766298,0.753478,0.753110,0.738153,0.693785,0.688408,Rectangle,Art,Font,Line,Gas,Symmetry,Pattern,Creative arts,Illustration,Wood,"['Rectangle', 'Art', 'Font', 'Line', 'Gas', 'S...","['0.8679025769233704', '0.8473606109619141', '...",222
3,3,"Work by @davidzinn in Ann Arbor, Michigan.",30,3289,https://scontent-atl3-1.cdninstagram.com/v/t51...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],10,0.882692,0.864480,0.863921,0.862133,0.850131,0.815706,0.815498,0.788973,0.748590,0.722193,Building,Rectangle,Brickwork,Brick,Building material,Wall,Line,Composite material,Pattern,Wood,"['Building', 'Rectangle', 'Brickwork', 'Brick'...","['0.8826923370361328', '0.8644804954528809', '...",98670
4,4,"“Touché, coulé” - new work by @matth.velvet in...",2,189,https://scontent-atl3-1.cdninstagram.com/v/t51...,4,0.695513,0.635542,0.555453,0.550535,NaN,NaN,NaN,NaN,NaN,NaN,Building,Building,Building,Building,NaN,NaN,NaN,NaN,NaN,NaN,"['Building', 'Building', 'Building', 'Building']","['0.6955127716064453', '0.6355423927307129', '...",10,0.944473,0.944195,0.935644,0.934243,0.897405,0.894920,0.894773,0.890608,0.856470,0.853654,Sky,Daytime,Window,Building,Paint,Azure,Art paint,Infrastructure,Urban design,Art,"['Sky', 'Daytime', 'Window', 'Building', 'Pain...","['0.9444730877876282', '0.9441952705383301', '...",378


In [ ]:
map_df.head()

,Caption,Comments,Likes,URL,Named Entities,Tokenized Entities,lat,lon,Name,Short Name,Country,Postal Code,Administrative Level 1,Administrative Level 1 Short,cleaned_captions,tokenized_captions,tagged_users
0,"@timo_levin wall in Kamianske, Ukraine 🇺🇦(2021...",11.0,2276.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,"['Kamianske', 'Ukraine']","[Kamianske, Ukraine]",48.523117,34.613683,Kamianske,Kamianske,Ukraine,51900,Dnipropetrovsk Oblast,Dnipropetrovsk Oblast,"@timo_levin wall in Kamianske, Ukraine 🇺🇦(2021...","['timo_levin', 'wall', 'Kamianske', ',', 'Ukra...",@timo_levin
1,"@jr wall in Paris, France 🇫🇷 (2021)#jr",46.0,6517.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,"['Paris', 'France']","[Paris, France]",48.856614,2.352222,Paris,Paris,France,None,Île-de-France,IDF,"@jr wall in Paris, France 🇫🇷 (2021)#jr","['jr', 'wall', 'Paris', ',', 'France', '🇫🇷', '...",@jr
2,"@3ttman wall in Lodz, Poland 🇵🇱(2013)#3ttman",37.0,5589.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,"['Lodz', 'Poland']","[Lodz, Poland]",51.759248,19.455983,Łódź,Łódź,Poland,91,Łódź Voivodeship,Łódź Voivodeship,"@3ttman wall in Lodz, Poland 🇵🇱(2013)#3ttman","['3ttman', 'wall', 'Lodz', ',', 'Poland', '🇵🇱'...",@3ttman
3,"@jessieandkatey wall in Philadelphia, USA 🇺🇸 (...",31.0,5188.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,"['Philadelphia', 'USA']","[Philadelphia, USA]",39.952584,-75.165222,Philadelphia,Philadelphia,United States,None,Pennsylvania,PA,"@jessieandkatey wall in Philadelphia, USA 🇺🇸 (...","['jessieandkatey', 'wall', 'Philadelphia', ','...",@jessieandkatey
4,"@romanlinacero wall in Nava de la Asunción, Se...",24.0,3584.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,"['Nava', 'de', 'la', 'Asunción', 'Segovia', 'S...","[Nava, de, la, Asunción, Segovia, Spain]",41.154601,-4.491565,Nava de la Asunción,Nava de la Asunción,Spain,40450,Castile and León,CL,"@romanlinacero wall in Nava de la Asunción, Se...","['romanlinacero', 'wall', 'Nava', 'de', 'la', ...",@romanlinacero


In [ ]:
hashtag_df.head()

,Username_ID,Caption,Comments,Comments_Text,Hashtags_Caption,Likes,Location,Mentions_Caption,Username,Named Entities,Tokenized Entities,lat,lon,Name,Short Name,Country,Postal Code,Administrative Level 1,Administrative Level 1 Short
0,175509226,Happy Friday! I wore this colorful kimono romp...,27,"['The cutest romper 😍', 'You’re in my neck of ...","['liketkit', 'ltkunder50', 'ltktravel', 'ltkcu...",46,"PostLocation(id=216296668, name='Galveston, Te...","['pinklily', 'gapsmack87', 'shipleydonuts_galv...",pearlsandpigsblog,"['Texas', 'texas']","[Texas, texas]",31.9685988,-99.9018131,Texas,TX,United States,None,Texas,TX
1,1599189434,#mrphotography #gainwithwestandmugweru #gainwi...,20,"['😍😍😍', '👏👏👏', '❤️', '❤️❤️❤️', '🔥🔥🔥', '🙌🙌', '❤...","['mrphotography', 'gainwithwestandmugweru', 'g...",20,"PostLocation(id=1760813094235018, name='Jhujha...",[],sameer.choudhary6375,[],[],None,None,None,None,None,None,None,None
2,2491624,@antoniopelayo x @isaacpelayo \n\nEdition of 5...,18,['#isaacpelayo #antoniopelayo #dope #love #art...,[],248,"PostLocation(id=212999109, name='Los Angeles, ...","['antoniopelayo', 'isaacpelayo']",isaacpelayo,"['Angeles', 'California']","[Angeles, California]",34.0522342,-118.2436849,Los Angeles,Los Angeles,United States,None,California,CA
3,47338304181,#digitalcollage #digitalartwork #digitalcollag...,8,"['🤩👌', '👏👏👏😍😍😍', 'Magnifique.👏👏👏👏👏', '🖤✨🤓✨🖤 Fa...","['digitalcollage', 'digitalartwork', 'digitalc...",90,NaN,[],thedi0nys0s,[],[],None,None,None,None,None,None,None,None
4,40846095404,#gainwithwestandmugweru #gainwithmchina#gainwi...,8,"['😘😘😘', '😍😍😍', '😍😍😍', '🔥🔥🔥', '😘😘😘', '❤️❤️❤️', ...","['gainwithwestandmugweru', 'gainwithmchina', '...",8,NaN,[],soyal_hamza,[],[],None,None,None,None,None,None,None,None


---

### Cleaning the Google vision dataframe to link it back to the pre-processed map DF

In [ ]:
posts_df.columns

Index(['Unnamed: 0', 'Caption', 'Comments', 'Likes', 'URL', 'Object_Num',
       'Object_0_score', 'Object_1_score', 'Object_2_score', 'Object_3_score',
       'Object_4_score', 'Object_5_score', 'Object_6_score', 'Object_7_score',
       'Object_8_score', 'Object_9_score', 'Object_0', 'Object_1', 'Object_2',
       'Object_3', 'Object_4', 'Object_5', 'Object_6', 'Object_7', 'Object_8',
       'Object_9', 'Total_Objects_Names', 'Total_Objects_Scores', 'Label_Num',
       'Label_0_score', 'Label_1_score', 'Label_2_score', 'Label_3_score',
       'Label_4_score', 'Label_5_score', 'Label_6_score', 'Label_7_score',
       'Label_8_score', 'Label_9_score', 'Label_0', 'Label_1', 'Label_2',
       'Label_3', 'Label_4', 'Label_5', 'Label_6', 'Label_7', 'Label_8',
       'Label_9', 'Labels_Objects_Names', 'Labels_Objects_Scores',
       'Engagement'],
      dtype='object')

In [ ]:
postsDF = posts_df[['Caption', 'Comments', 'Likes', 'URL', 'Object_Num', 'Label_0', 'Label_1', 'Label_2','Engagement']]

In [ ]:
map_df.columns

Index(['Caption', 'Comments', 'Likes', 'URL', 'Named Entities',
       'Tokenized Entities', 'lat', 'lon', 'Name', 'Short Name', 'Country',
       'Postal Code', 'Administrative Level 1', 'Administrative Level 1 Short',
       'cleaned_captions', 'tokenized_captions', 'tagged_users'],
      dtype='object')

In [ ]:
mapDF = map_df[['URL', 'lat', 'lon', 'Short Name', 'Country', 'tagged_users']]

In [ ]:
## Merging the two dataframes
mergedDF = postsDF.merge(mapDF, on='URL', how='right')
mergedDF.head()

,Caption,Comments,Likes,URL,Object_Num,Label_0,Label_1,Label_2,Engagement,lat,lon,Short Name,Country,tagged_users
0,"@timo_levin wall in Kamianske, Ukraine 🇺🇦(2021...",11.0,2276.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,1.0,Sky,Cloud,Building,25036.0,48.523117,34.613683,Kamianske,Ukraine,@timo_levin
1,"@jr wall in Paris, France 🇫🇷 (2021)\n•\n#jr #u...",46.0,6517.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,4.0,Cloud,Sky,Building,299782.0,48.856614,2.352222,Paris,France,@jr
2,"@3ttman wall in Lodz, Poland 🇵🇱(2013)\n•\n#3tt...",37.0,5589.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,3.0,Cloud,Daytime,Sky,206793.0,51.759248,19.455983,Łódź,Poland,@3ttman
3,"@jessieandkatey wall in Philadelphia, USA 🇺🇸 (...",31.0,5188.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,2.0,Cloud,Sky,Building,160828.0,39.952584,-75.165222,Philadelphia,United States,@jessieandkatey
4,"@romanlinacero wall in Nava de la Asunción, Se...",24.0,3584.0,https://scontent-bos3-1.cdninstagram.com/v/t51...,7.0,Sky,Paint,Art,86016.0,41.154601,-4.491565,Nava de la Asunción,Spain,@romanlinacero


In [ ]:
mergedDF['lat'].isnull().value_counts() ## this confirms that all rows have longitude and latitude

False    1011
Name: lat, dtype: int64

---

In [ ]:
#Looking at the hashtag DF
hashtag_df.columns

Index(['Username_ID', 'Caption', 'Comments', 'Comments_Text',
       'Hashtags_Caption', 'Likes', 'Location', 'Mentions_Caption', 'Username',
       'Named Entities', 'Tokenized Entities', 'lat', 'lon', 'Name',
       'Short Name', 'Country', 'Postal Code', 'Administrative Level 1',
       'Administrative Level 1 Short'],
      dtype='object')

In [ ]:
hashtag_df.head()

,Username_ID,Caption,Comments,Comments_Text,Hashtags_Caption,Likes,Location,Mentions_Caption,Username,Named Entities,Tokenized Entities,lat,lon,Name,Short Name,Country,Postal Code,Administrative Level 1,Administrative Level 1 Short
0,175509226,Happy Friday! I wore this colorful kimono romp...,27,"['The cutest romper 😍', 'You’re in my neck of ...","['liketkit', 'ltkunder50', 'ltktravel', 'ltkcu...",46,"PostLocation(id=216296668, name='Galveston, Te...","['pinklily', 'gapsmack87', 'shipleydonuts_galv...",pearlsandpigsblog,"['Texas', 'texas']","[Texas, texas]",31.9685988,-99.9018131,Texas,TX,United States,None,Texas,TX
1,1599189434,#mrphotography #gainwithwestandmugweru #gainwi...,20,"['😍😍😍', '👏👏👏', '❤️', '❤️❤️❤️', '🔥🔥🔥', '🙌🙌', '❤...","['mrphotography', 'gainwithwestandmugweru', 'g...",20,"PostLocation(id=1760813094235018, name='Jhujha...",[],sameer.choudhary6375,[],[],None,None,None,None,None,None,None,None
2,2491624,@antoniopelayo x @isaacpelayo \n\nEdition of 5...,18,['#isaacpelayo #antoniopelayo #dope #love #art...,[],248,"PostLocation(id=212999109, name='Los Angeles, ...","['antoniopelayo', 'isaacpelayo']",isaacpelayo,"['Angeles', 'California']","[Angeles, California]",34.0522342,-118.2436849,Los Angeles,Los Angeles,United States,None,California,CA
3,47338304181,#digitalcollage #digitalartwork #digitalcollag...,8,"['🤩👌', '👏👏👏😍😍😍', 'Magnifique.👏👏👏👏👏', '🖤✨🤓✨🖤 Fa...","['digitalcollage', 'digitalartwork', 'digitalc...",90,NaN,[],thedi0nys0s,[],[],None,None,None,None,None,None,None,None
4,40846095404,#gainwithwestandmugweru #gainwithmchina#gainwi...,8,"['😘😘😘', '😍😍😍', '😍😍😍', '🔥🔥🔥', '😘😘😘', '❤️❤️❤️', ...","['gainwithwestandmugweru', 'gainwithmchina', '...",8,NaN,[],soyal_hamza,[],[],None,None,None,None,None,None,None,None


In [ ]:
hashtagDF = hashtag_df[['Caption', 'Comments', 'Likes', 'Username', 'lat', 'lon', 'Short Name', 'Country']]

In [ ]:
hashtagDF['Caption'] = hashtagDF['Caption'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
## Detecting tagged users in hashtag dataset
tagged_users = []
I = range(len(hashtagDF))

for i in I:
  full_string = hashtagDF.loc[i, "Caption"]
  string_caption = full_string.split()
  for j in string_caption:
    if j.startswith("@"):
      tagged = j
  if full_string.startswith("@") == True:
    tagged_users.append(tagged)
  else:
    tagged_users.append('no mention')

In [ ]:
hashtagDF['tagged_Users'] = tagged_users

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
hashtagDF.head()

,Caption,Comments,Likes,Username,lat,lon,Short Name,Country,tagged_Users
0,Happy Friday! I wore this colorful kimono romp...,27,46,pearlsandpigsblog,31.9685988,-99.9018131,TX,United States,no mention
1,#mrphotography #gainwithwestandmugweru #gainwi...,20,20,sameer.choudhary6375,None,None,None,None,no mention
2,@antoniopelayo x @isaacpelayo \n\nEdition of 5...,18,248,isaacpelayo,34.0522342,-118.2436849,Los Angeles,United States,@isaacpelayo
3,#digitalcollage #digitalartwork #digitalcollag...,8,90,thedi0nys0s,None,None,None,None,no mention
4,#gainwithwestandmugweru #gainwithmchina#gainwi...,8,8,soyal_hamza,None,None,None,None,no mention


In [ ]:
#Removing posts without location
hashtagDF.dropna(subset = ["lat"], inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
hashtagDF = hashtagDF[hashtagDF['lat'] != 'None']

In [ ]:
hashtagDF.head()

,Caption,Comments,Likes,Username,lat,lon,Short Name,Country,tagged_Users
0,Happy Friday! I wore this colorful kimono romp...,27,46,pearlsandpigsblog,31.9685988,-99.9018131,TX,United States,no mention
2,@antoniopelayo x @isaacpelayo \n\nEdition of 5...,18,248,isaacpelayo,34.0522342,-118.2436849,Los Angeles,United States,@isaacpelayo
5,#balochisong💙💚❤ #karachipakistan🇵🇰 \n#kingofmy...,7,29,maryam_baloshi2,37.5048794,-122.2594344,716,United States,no mention
6,קפצה להגיד שלום בסט @shaharlaz\nפוסט אחרון למא...,6,24,sapir_anaki88,32.0852999,34.78176759999999,Tel Aviv-Yafo,Israel,no mention
9,Mural 3: Iron Paradise\n.\nAnother big paintin...,5,17,clarkdef,39.6356048,-84.931071,Liberty,United States,no mention


---

## Exporting dataframes

In [ ]:
from google.colab import files
mergedDF.to_csv('mergedDF.csv', index = False) 
files.download('mergedDF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
hashtagDF.to_csv('hashtagDF.csv', index = False) 
files.download('hashtagDF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>